In [86]:
import pandas as pd
import numpy as np
import math

In [87]:
predf = pd.read_csv('../../data/usa_00004.csv.gz')

In [88]:
predf =  predf.loc[predf['PERNUM'] == 1]

In [89]:
predf = predf.drop(['SAMPLE', 'SERIAL', 'CBSERIAL', 'CLUSTER', 'COUNTYICP','STRATA','PERNUM_HEAD', 'GQ', 'RACED', 'PERWT'], axis = 1)

In [90]:
predf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88979 entries, 0 to 210221
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   YEAR       88979 non-null  int64  
 1   HHWT       88979 non-null  float64
 2   STATEFIP   88979 non-null  int64  
 3   COUNTYFIP  88979 non-null  int64  
 4   PERNUM     88979 non-null  int64  
 5   SEX        88979 non-null  int64  
 6   AGE        88979 non-null  int64  
 7   MARST      88979 non-null  int64  
 8   RACE       88979 non-null  int64  
 9   HCOVANY    88979 non-null  int64  
 10  EDUC       88979 non-null  int64  
 11  EDUCD      88979 non-null  int64  
 12  EMPSTAT    88979 non-null  int64  
 13  EMPSTATD   88979 non-null  int64  
 14  FTOTINC    88979 non-null  int64  
dtypes: float64(1), int64(14)
memory usage: 10.9 MB


In [91]:
predf.HCOVANY.replace({1:0, 2:1}, inplace=True)

In [92]:
predf.SEX.replace({1:0, 2:1}, inplace=True)

In [93]:
predf

,YEAR,HHWT,STATEFIP,COUNTYFIP,PERNUM,SEX,AGE,MARST,RACE,HCOVANY,EDUC,EDUCD,EMPSTAT,EMPSTATD,FTOTINC
0,2015,838.32,6,37,1,1,28,6,1,1,7,71,1,10,40730
2,2015,1092.81,6,73,1,1,78,5,1,1,7,71,1,10,24600
3,2015,913.17,6,85,1,0,24,6,1,0,7,71,1,10,25000
6,2015,868.26,6,1,1,0,30,1,6,1,10,101,1,10,162300
10,2015,1556.88,6,85,1,1,63,4,1,1,6,65,1,10,122000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210213,2019,92.88,48,309,1,0,57,1,1,1,10,101,1,10,119000
210215,2019,1269.36,48,453,1,1,32,1,1,1,6,64,1,10,79000
210219,2019,681.12,48,439,1,1,66,5,1,1,11,114,3,30,77000
210220,2019,1904.04,48,0,1,0,73,5,1,1,7,71,3,30,25300


In [94]:
predf['COUNTYFIP'] = predf.COUNTYFIP.astype(str).str.zfill(3)

In [95]:
predf.HHWT = round(predf['HHWT']).astype(int)

In [96]:
df = predf

In [97]:
def YearStateDF(df, year, statefip):
    dfstate = df.loc[(df['YEAR'] == year) & (df['STATEFIP'] == statefip)]
    dfstatewt = dfstate.loc[dfstate.index.repeat(dfstate.HHWT)]
    newdf = dfstatewt.set_index('COUNTYFIP')
    return newdf

In [118]:
def NumericDF(df):
    inc = pd.DataFrame(df.groupby('COUNTYFIP')['FTOTINC'].mean().round(decimals=2))
    df['avg_inc'] = inc['FTOTINC']
    age = pd.DataFrame(df.groupby('COUNTYFIP')['AGE'].mean().round(decimals=2))
    df['avg_age'] = age['AGE']
    educ = pd.DataFrame(df.groupby('COUNTYFIP')['EDUC'].mean().round(decimals=2))
    df['avg_educ'] = educ['EDUC']
    educd = pd.DataFrame(df.groupby('COUNTYFIP')['EDUCD'].mean().round(decimals=2))
    df['avg_educd'] = educd['EDUCD']
    sex = pd.DataFrame(df.groupby('COUNTYFIP')['SEX'].mean().round(decimals=2))
    df['sex'] = sex['SEX']
    insurance = pd.DataFrame(df.groupby('COUNTYFIP')['HCOVANY'].mean().round(decimals=2))
    df['insurance'] = insurance['HCOVANY']
    df = df.drop(['FTOTINC', 'AGE', 'EDUC', 'EDUCD', 'HHWT', 'SEX', 'HCOVANY','PERNUM','MARST','RACE','EMPSTAT','EMPSTATD'], axis = 1)
    df = df.drop_duplicates()
    return df

In [100]:
marst_mapper ={1:'married_spouse_present',
2:'married_spouse_absent',
3:'separated',
4:'divorced',
5:'widowed',
6:'single'}

In [101]:
emp_mapper = {0:'NA',
10:'at_work',
12:'has_job_not_working',
14:'armed_forces_at_work',
15:'armed_forces_not_at_work_but_with_job',
20:'unemployed',
30:'not_in_labor_force'}

In [102]:
race_mapper = {1:'White',
2:'Black/African American/Negro',
3:'American Indian or Alaska Native',
4:'Chinese',
5:'Japanese',
6:'Other Asian or Pacific Islander',
7:'Other race, nec',
8:'Two major races',
9:'Three or more major races',}

In [103]:
emp_list = ['NA','at_work','has_job_not_working','armed_forces_at_work','armed_forces_not_at_work_but_with_job','unemployed','not_in_labor_force']

In [104]:
race_list = ['white',
'black',
'american_indian_or_alaska_native',
'chinese',
'japanese',
'other_asian_or_pacific_islander',
'other_race',
'two_major_races',
'three_or_more_major_races']

In [105]:
marst_list = ['married_spouse_present','married_spouse_absent','separated','divorced','widowed','single']

In [106]:
column_list = ['RACE', 'EMPTSTATD', 'MARST']
dict_list = [race_mapper, emp_mapper, marst_mapper]
list_list = [race_list, emp_list, marst_list]

In [55]:
def catmap(df, column, dict, list):
    df[column].map(dict)
    dfcatmap = df.reset_index().groupby(['COUNTYFIP', column]).count().reset_index().pivot(columns=column, 
                                                                                             index='COUNTYFIP',
                                                                                                values='YEAR')
    dfcatmap.columns = list
    dfcatmap.fillna(0, inplace=True)
    return dfcatmap

In [23]:
#pd.DataFrame(df.groupby(['STATEFIP', 'COUNTYFIP']).count().T.sum()).reset_index().groupby('STATEFIP').sum()

In [108]:
CA2015 = YearStateDF(df, 2015, 6)
CA2015

,YEAR,HHWT,STATEFIP,PERNUM,SEX,AGE,MARST,RACE,HCOVANY,EDUC,EDUCD,EMPSTAT,EMPSTATD,FTOTINC
COUNTYFIP,,,,,,,,,,,,,,
037,2015,838,6,1,1,28,6,1,1,7,71,1,10,40730
037,2015,838,6,1,1,28,6,1,1,7,71,1,10,40730
037,2015,838,6,1,1,28,6,1,1,7,71,1,10,40730
037,2015,838,6,1,1,28,6,1,1,7,71,1,10,40730
037,2015,838,6,1,1,28,6,1,1,7,71,1,10,40730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
037,2015,913,6,1,0,44,1,6,1,11,114,2,20,4500
037,2015,913,6,1,0,44,1,6,1,11,114,2,20,4500
037,2015,913,6,1,0,44,1,6,1,11,114,2,20,4500


In [112]:
CA2015.SEX.mean()

0.4912906707171701

In [111]:
CA2015.HCOVANY.value_counts()

1    12755327
0     1124100
Name: HCOVANY, dtype: int64

In [119]:
CA2015NUM = NumericDF(CA2015)
CA2015NUM

,YEAR,STATEFIP,avg_inc,avg_age,avg_educ,avg_educd,sex,insurance
COUNTYFIP,,,,,,,,
037,2015,6,527508.44,50.78,7.28,75.04,0.49,0.90
073,2015,6,765623.18,49.56,7.63,78.59,0.49,0.91
085,2015,6,615511.78,49.99,8.28,85.24,0.46,0.96
001,2015,6,500963.46,49.29,8.08,83.08,0.47,0.93
029,2015,6,1244773.84,45.40,5.84,60.66,0.51,0.86
000,2015,6,980584.96,53.42,7.07,72.98,0.49,0.90
077,2015,6,479714.11,49.58,6.96,71.74,0.47,0.93
067,2015,6,543471.23,47.84,7.43,76.39,0.51,0.95
065,2015,6,446909.92,51.03,6.85,70.86,0.51,0.86


In [49]:
CA2015cat1 = catmap(CA2015, 'RACE', race_mapper, race_list)

In [58]:
CA2015cat2 = catmap(CA2015, 'MARST', marst_mapper, marst_list)

In [67]:
CA_emp_list = ['NA','at_work','has_job_not_working','armed_forces_at_work','unemployed','not_in_labor_force']

In [68]:
def catmap(df, column, dict, list):
    df[column].map(dict)
    dfcatmap = df.reset_index().groupby(['COUNTYFIP', column]).count().reset_index().pivot(columns=column, 
                                                                                             index='COUNTYFIP',
                                                                                                values='YEAR')
    dfcatmap.columns = list
    dfcatmap.fillna(0, inplace=True)
    return dfcatmap

In [71]:
CA2015cat3 = catmap(CA2015, 'EMPSTATD', emp_mapper, CA_emp_list)
CA2015cat3

,NA,at_work,has_job_not_working,armed_forces_at_work,unemployed,not_in_labor_force
COUNTYFIP,,,,,,
000,180.0,263317.0,7170.0,9295.0,16691.0,269472.0
001,0.0,380739.0,21226.0,913.0,20312.0,175616.0
007,0.0,44701.0,2425.0,0.0,6033.0,48128.0
013,0.0,234025.0,14163.0,0.0,9761.0,125539.0
017,0.0,37994.0,0.0,0.0,2635.0,37500.0
019,0.0,175661.0,5120.0,0.0,12485.0,118771.0
023,0.0,18848.0,1751.0,0.0,958.0,25419.0
025,0.0,30133.0,554.0,0.0,434.0,20134.0
029,0.0,162154.0,3443.0,1093.0,22860.0,124874.0


In [121]:
CA2015CONCAT = pd.concat([CA2015NUM, CA2015cat1, CA2015cat2, CA2015cat3], axis=1)
CA2015CONCAT

,YEAR,STATEFIP,avg_inc,avg_age,avg_educ,avg_educd,sex,insurance,white,black,...,separated,divorced,widowed,single,NA,at_work,has_job_not_working,armed_forces_at_work,unemployed,not_in_labor_force
037,2015,6,527508.44,50.78,7.28,75.04,0.49,0.90,1927771.0,377965.0,...,156807.0,427986.0,281459.0,1091137.0,1168.0,2231731.0,65418.0,1752.0,94430.0,1128328.0
073,2015,6,765623.18,49.56,7.63,78.59,0.49,0.91,908667.0,65988.0,...,20434.0,230323.0,91125.0,318545.0,60.0,715534.0,10000.0,49147.0,34418.0,376396.0
085,2015,6,615511.78,49.99,8.28,85.24,0.46,0.96,356897.0,26871.0,...,23578.0,78472.0,51407.0,184357.0,764.0,453654.0,13219.0,0.0,19641.0,261507.0
001,2015,6,500963.46,49.29,8.08,83.08,0.47,0.93,288719.0,91350.0,...,15777.0,79355.0,35150.0,151388.0,0.0,380739.0,21226.0,913.0,20312.0,175616.0
029,2015,6,1244773.84,45.40,5.84,60.66,0.51,0.86,225788.0,27155.0,...,12243.0,44160.0,18081.0,83158.0,0.0,162154.0,3443.0,1093.0,22860.0,124874.0
000,2015,6,980584.96,53.42,7.07,72.98,0.49,0.90,456023.0,21810.0,...,6647.0,99534.0,63950.0,102472.0,180.0,263317.0,7170.0,9295.0,16691.0,269472.0
077,2015,6,479714.11,49.58,6.96,71.74,0.47,0.93,146226.0,6692.0,...,4416.0,35913.0,17513.0,44940.0,30.0,123937.0,9341.0,2260.0,12875.0,74622.0
067,2015,6,543471.23,47.84,7.43,76.39,0.51,0.95,330111.0,56122.0,...,10509.0,75658.0,29027.0,149910.0,0.0,327957.0,8997.0,0.0,14296.0,182755.0
065,2015,6,446909.92,51.03,6.85,70.86,0.51,0.86,513175.0,46464.0,...,33099.0,108050.0,69613.0,101500.0,0.0,417076.0,8322.0,1497.0,25809.0,274777.0
013,2015,6,387181.29,54.22,8.12,83.42,0.57,0.96,242630.0,42157.0,...,7216.0,62095.0,28563.0,63937.0,0.0,234025.0,14163.0,0.0,9761.0,125539.0


In [134]:
#CA2015CONCAT['FIP'] = list(zip(CA2015CONCAT['STATEFIP'], CA2015CONCAT.index ))

CA2015CONCAT['FIP'] = CA2015CONCAT['STATEFIP'].astype(str) + CA2015CONCAT.index.astype(str)

In [135]:
CA2015CONCAT

,YEAR,STATEFIP,avg_inc,avg_age,avg_educ,avg_educd,sex,insurance,white,black,...,divorced,widowed,single,NA,at_work,has_job_not_working,armed_forces_at_work,unemployed,not_in_labor_force,FIP
037,2015,6,527508.44,50.78,7.28,75.04,0.49,0.90,1927771.0,377965.0,...,427986.0,281459.0,1091137.0,1168.0,2231731.0,65418.0,1752.0,94430.0,1128328.0,6037
073,2015,6,765623.18,49.56,7.63,78.59,0.49,0.91,908667.0,65988.0,...,230323.0,91125.0,318545.0,60.0,715534.0,10000.0,49147.0,34418.0,376396.0,6073
085,2015,6,615511.78,49.99,8.28,85.24,0.46,0.96,356897.0,26871.0,...,78472.0,51407.0,184357.0,764.0,453654.0,13219.0,0.0,19641.0,261507.0,6085
001,2015,6,500963.46,49.29,8.08,83.08,0.47,0.93,288719.0,91350.0,...,79355.0,35150.0,151388.0,0.0,380739.0,21226.0,913.0,20312.0,175616.0,6001
029,2015,6,1244773.84,45.40,5.84,60.66,0.51,0.86,225788.0,27155.0,...,44160.0,18081.0,83158.0,0.0,162154.0,3443.0,1093.0,22860.0,124874.0,6029
000,2015,6,980584.96,53.42,7.07,72.98,0.49,0.90,456023.0,21810.0,...,99534.0,63950.0,102472.0,180.0,263317.0,7170.0,9295.0,16691.0,269472.0,6000
077,2015,6,479714.11,49.58,6.96,71.74,0.47,0.93,146226.0,6692.0,...,35913.0,17513.0,44940.0,30.0,123937.0,9341.0,2260.0,12875.0,74622.0,6077
067,2015,6,543471.23,47.84,7.43,76.39,0.51,0.95,330111.0,56122.0,...,75658.0,29027.0,149910.0,0.0,327957.0,8997.0,0.0,14296.0,182755.0,6067
065,2015,6,446909.92,51.03,6.85,70.86,0.51,0.86,513175.0,46464.0,...,108050.0,69613.0,101500.0,0.0,417076.0,8322.0,1497.0,25809.0,274777.0,6065
013,2015,6,387181.29,54.22,8.12,83.42,0.57,0.96,242630.0,42157.0,...,62095.0,28563.0,63937.0,0.0,234025.0,14163.0,0.0,9761.0,125539.0,6013


In [141]:
CA2015CONCAT.to_csv('2015_California_Census_Data.csv')

In [128]:
CA2015CONCAT['FIP'].astype(str)

037    (6, '037')
073    (6, '073')
085    (6, '085')
001    (6, '001')
029    (6, '029')
000    (6, '000')
077    (6, '077')
067    (6, '067')
065    (6, '065')
013    (6, '013')
083    (6, '083')
075    (6, '075')
019    (6, '019')
071    (6, '071')
059    (6, '059')
061    (6, '061')
097    (6, '097')
095    (6, '095')
039    (6, '039')
099    (6, '099')
031    (6, '031')
089    (6, '089')
113    (6, '113')
025    (6, '025')
023    (6, '023')
111    (6, '111')
079    (6, '079')
081    (6, '081')
107    (6, '107')
007    (6, '007')
017    (6, '017')
055    (6, '055')
087    (6, '087')
041    (6, '041')
047    (6, '047')
Name: FIP, dtype: object

In [123]:
CA2015CONCAT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 037 to 047
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   YEAR                              35 non-null     int64  
 1   STATEFIP                          35 non-null     int64  
 2   avg_inc                           35 non-null     float64
 3   avg_age                           35 non-null     float64
 4   avg_educ                          35 non-null     float64
 5   avg_educd                         35 non-null     float64
 6   sex                               35 non-null     float64
 7   insurance                         35 non-null     float64
 8   white                             35 non-null     float64
 9   black                             35 non-null     float64
 10  american_indian_or_alaska_native  35 non-null     float64
 11  chinese                           35 non-null     float64
 12  japanese    

In [ ]:
df.STATEFIP.unique()

In [ ]:
#df.STATEFIP.value_counts()

In [ ]:
TX2015 = YearStateDF(df, 2015, 48)
TX2015

In [ ]:
#TX2015 = YearStateDf(df, 2015, 48)
#TX2015

In [ ]:
TX2015 = NumericDF(TX2015)
TX2015

In [ ]:
TX2015['RACE'].value_counts()

In [ ]:
predf.EMPSTATD.value_counts()

In [ ]:
emp_mapper = {0:'NA',
10:'at_work',
12:'has_job_not_working',
14:'armed_forces_at_work',
15:'armed_forces_not_at_work_but_with_job',
20:'unemployed',
30:'not_in_labor_force'}

In [ ]:
emp_list = ['NA','at_work','has_job_not_working','armed_forces_at_work','armed_forces_not_at_work_but_with_job','unemployed','not_in_labor_force']

In [ ]:
#def catmap(df, column, dict):
   # df[column].map(dict)
   # dfcatmap = df.reset_index().groupby(['COUNTYFIP', column]).count().reset_index().pivot(columns=column, 
                                                                                            #index='COUNTYFIP',
                                                                                            #values='YEAR')
   # return dfcatmap

In [57]:
txempstatd = catmap(TX2015, 'EMPSTATD', emp_mapper, emp_list)
txempstatd

NameError: name 'TX2015' is not defined

In [ ]:
race_mapper = {1:'White',
2:'Black/African American/Negro',
3:'American Indian or Alaska Native',
4:'Chinese',
5:'Japanese',
6:'Other Asian or Pacific Islander',
7:'Other race, nec',
8:'Two major races',
9:'Three or more major races',}

TX2015['RACE'].map(race_mapper)

In [ ]:
race_list = ['white',
'black',
'american_indian_or_alaska_native',
'chinese',
'japanese',
'other_asian_or_pacific_islander',
'other_race',
'two_major_races',
'three_or_more_major_races']

In [ ]:
txrace= catmap(TX2015, 'RACE', race_mapper, race_list)
txrace

In [ ]:
#TXbyRACE = TX2015.reset_index().groupby(['COUNTYFIP', 'RACE']).count().reset_index().pivot(columns='RACE', 
                                                                                       #  index='COUNTYFIP',
                                                                                       # values='YEAR')
#TXbyRACE

In [ ]:
marst_list = ['married_spouse_present','married_spouse_absent','separated','divorced','widowed','single']

In [ ]:
marst_mapper ={1:'married_spouse_present',
2:'married_spouse_absent',
3:'separated',
4:'divorced',
5:'widowed',
6:'single'}
TX2015['MARST'].map(marital_mapper)

In [ ]:
TX2015['MARST'].value_counts()

In [ ]:
TXMARST = TX2015.reset_index().groupby(['COUNTYFIP', 'MARST']).count().reset_index().pivot(columns='MARST', 
                                                                                         index='COUNTYFIP',
                                                                                        values='YEAR')
TXMARST

In [ ]:
numerics = ['AGE', 'EDUC', 'EDUCD', 'FTOTINC']
categoricals = ['SEX', 'MARST','RACE','EMPSTAT', 'EMPSTATD']
binary = ['HCOVANY','SEX']

In [ ]:
#I need to make 2 functions
#one that makes a dataframe by state and year
#one that makes a new dataframe grouped by colums with means of numerical data and dummies of categorical data?

In [ ]:
#def YearStateDf(df, year, statefip):
 #   dfstate = df.loc[(df['YEAR'] == year) & (df['STATEFIP'] == statefip)]
    #dfstatewt = dfstate.loc[dfstate.index.repeat(dfstate.HHWT)]
    #dfstatewt.set_index('COUNTYFIP')
    #return dfstatewt

In [ ]:
#def Numeric(df):
    #inc = pd.DataFrame(df.groupby('COUNTYFIP')['FTOTINC'].mean().round(decimals=2))
    #df['avg_inc'] = inc['FTOTINC']
    #age = pd.DataFrame(df.groupby('COUNTYFIP')['AGE'].mean()).round(decimals=2))
    #df['avg_age'] = age['AGE']
    #return df